In [3]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()
Faker.seed(100)
random.seed(100)

# Configuration
num_clients = 1000
avg_transactions_per_day = 5
days = 90  # roughly 3 months
loan_types = ["Home Loan", "Auto Loan", "Personal Loan"]
start_date = datetime(2023, 1, 1)

clients = []
transactions = []
loans = []

for _ in range(num_clients):
    client_id = fake.uuid4()
    age = random.randint(25, 60)
    annual_income = random.randint(40000, 180000)
    monthly_salary = round(annual_income / 12, 2)
    loan_type = random.choice(loan_types)

    clients.append({
        "client_id": client_id,
        "age": age,
        "annual_income": annual_income,
        "loan_type": loan_type
    })

    loan_amount = random.randint(10000, 500000)
    emi = round(loan_amount / random.randint(12, 60), 2)
    loans.append({
        "client_id": client_id,
        "loan_type": loan_type,
        "loan_amount": loan_amount,
        "emi": emi
    })

    for d in range(days):
        date = start_date + timedelta(days=d)

        # Salary once a month
        if d % 30 == 0:
            transactions.append({
                "client_id": client_id,
                "date": date,
                "amount": monthly_salary,
                "type": "credit",
                "category": "salary"
            })

        # EMI once a month
        if d % 30 == 5:
            transactions.append({
                "client_id": client_id,
                "date": date,
                "amount": -emi,
                "type": "debit",
                "category": "emi"
            })

        # 3–7 random debits per day
        for _ in range(random.randint(3, 7)):
            amount = round(random.uniform(10, 300), 2)
            category = random.choice(["groceries", "utilities", "entertainment", "shopping", "fuel", "subscriptions"])
            transactions.append({
                "client_id": client_id,
                "date": date,
                "amount": -amount,
                "type": "debit",
                "category": category
            })

clients_df = pd.DataFrame(clients)
transactions_df = pd.DataFrame(transactions)
loans_df = pd.DataFrame(loans)

clients_df.head(), transactions_df.head(), loans_df.head()


(                              client_id  age  annual_income      loan_type
 0  f3a3c571-7476-4899-b5a3-adb3254a9493   34         160462      Auto Loan
 1  649dda6e-b49c-43dc-acbc-408cc5521660   45          72662      Home Loan
 2  81c1e7ff-6efa-4d5b-9988-5afcbb61a9cd   51         123864  Personal Loan
 3  1f0e4b4a-886c-4a30-9c26-ffa8ccce240c   46         123296      Home Loan
 4  436f40f2-74b8-4e87-bc99-1c531484f407   33          53462      Home Loan,
                               client_id       date    amount    type  \
 0  f3a3c571-7476-4899-b5a3-adb3254a9493 2023-01-01  13371.83  credit   
 1  f3a3c571-7476-4899-b5a3-adb3254a9493 2023-01-01   -222.27   debit   
 2  f3a3c571-7476-4899-b5a3-adb3254a9493 2023-01-01   -156.99   debit   
 3  f3a3c571-7476-4899-b5a3-adb3254a9493 2023-01-01   -164.54   debit   
 4  f3a3c571-7476-4899-b5a3-adb3254a9493 2023-01-01   -223.65   debit   
 
         category  
 0         salary  
 1       shopping  
 2      groceries  
 3      groceries  
 4 

In [2]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
clients_df.count()

client_id        1000
age              1000
annual_income    1000
loan_type        1000
dtype: int64

In [8]:
from neo4j import GraphDatabase

def connect_to_neo4j(uri, username, password):
    """
    Establishes a connection to a Neo4j database.

    Args:
        uri (str): The URI of the Neo4j database (e.g., "bolt://localhost:7687").
        username (str): The username for authentication.
        password (str): The password for authentication.

    Returns:
        neo4j.Session: A Neo4j session object, or None if the connection fails.
    """
    driver = None
    try:
        driver = GraphDatabase.driver(uri, auth=(username, password))
        return driver.session()  # Return a session, not the driver
    except Exception as e:
        print(f"Error connecting to Neo4j: {e}")
        if driver:
            driver.close()
        return None

def close_neo4j_connection(session):
    """
    Closes the Neo4j session.

    Args:
        session (neo4j.Session): The Neo4j session to close.
    """
    if session:
        try:
            session.close()
        except Exception as e:
            print(f"Error closing Neo4j session: {e}")

if __name__ == "__main__":
    # Replace with your actual Neo4j connection details
    uri = "neo4j+s://e14248e5.databases.neo4j.io"  # Or "neo4j://localhost:7687"
    username = "neo4j"
    password = "j_OfoqhHCpE1vq-qFEHfcyI0WkjIKcGktFmqGtnp0Oc"

    session = connect_to_neo4j(uri, username, password)
    if session:
        print("Successfully connected to Neo4j!")

        try:
            # Example: Run a simple Cypher query
            result = session.run("MATCH (n:Client) RETURN n LIMIT 5")  # Corrected query
            for record in result:
                print(record)  # Print the record

        except Exception as e:
            print(f"Error executing query: {e}")
        finally:
            close_neo4j_connection(session) # Close session in a finally block
    else:
        print("Failed to connect to Neo4j.")

Successfully connected to Neo4j!


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Client)} {position: line: 1, column: 10, offset: 9} for query: 'MATCH (n:Client) RETURN n LIMIT 5'


In [6]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [9]:
from neo4j import GraphDatabase
import pandas as pd

def load_client_data_from_dataframe(session, df):
    """
    Loads client data from a Pandas DataFrame into Neo4j.

    Args:
        session (neo4j.Session): The Neo4j session to use.
        df (pd.DataFrame): The Pandas DataFrame containing client data
                          with columns: 'client_id', 'age', 'annual_income', 'loan_type'.
    """
    for _, row in df.iterrows():
        query = """
        CREATE (c:Client {
          client_id: $client_id,
          age: toInteger($age),
          annual_income: toFloat($annual_income),
          loan_type: $loan_type
        })
        """
        parameters = {
            "client_id": row["client_id"],
            "age": row["age"],
            "annual_income": row["annual_income"],
            "loan_type": row["loan_type"],
        }
        session.run(query, parameters)

def load_transaction_data_from_dataframe(session, df):
    """
    Loads client data from a Pandas DataFrame into Neo4j.

    Args:
        session (neo4j.Session): The Neo4j session to use.
        df (pd.DataFrame): The Pandas DataFrame containing client data
                          with columns: 'client_id', 'age', 'annual_income', 'loan_type'.
    """
    for _, row in df.iterrows():
        query = """
        
        """
        parameters = {
            "client_id": row["client_id"],
            "age": row["age"],
            "annual_income": row["annual_income"],
            "loan_type": row["loan_type"],
        }
        session.run(query, parameters)
        
if __name__ == "__main__":
    # Replace with your actual Neo4j connection details
    uri = "neo4j+s://e14248e5.databases.neo4j.io"  # Or "neo4j://localhost:7687"
    username = "neo4j"
    password = "j_OfoqhHCpE1vq-qFEHfcyI0WkjIKcGktFmqGtnp0Oc"


    session = connect_to_neo4j(uri, username, password)
    if session:
        print("Successfully connected to Neo4j!")
        try:
            load_client_data_from_dataframe(session, clients_df)
            print("Client data loaded successfully from DataFrame!")

            # load_transaction_data_from_dataframe(session, transactions_df)
            # print("Transaction data loaded successfully from DataFrame!")

            # load_loan_data_from_dataframe(session, loans_df)
            # print("Loan data loaded successfully from DataFrame!")            

            # Example: Run a simple Cypher query after loading the data
            result = session.run("MATCH (c:Client) RETURN c LIMIT 5")
            for record in result:
                print(record)

        except Exception as e:
            print(f"Error: {e}")
        finally:
            close_neo4j_connection(session)
    else:
        print("Failed to connect to Neo4j.")


Successfully connected to Neo4j!
Client data loaded successfully from DataFrame!
<Record c=<Node element_id='4:7ee77186-f382-4f2a-ad67-d6d490e9527b:0' labels=frozenset({'Client'}) properties={'loan_type': 'Auto Loan', 'client_id': 'f3a3c571-7476-4899-b5a3-adb3254a9493', 'age': 34, 'annual_income': 160462.0}>>
<Record c=<Node element_id='4:7ee77186-f382-4f2a-ad67-d6d490e9527b:1' labels=frozenset({'Client'}) properties={'loan_type': 'Home Loan', 'client_id': '649dda6e-b49c-43dc-acbc-408cc5521660', 'age': 45, 'annual_income': 72662.0}>>
<Record c=<Node element_id='4:7ee77186-f382-4f2a-ad67-d6d490e9527b:2' labels=frozenset({'Client'}) properties={'loan_type': 'Personal Loan', 'client_id': '81c1e7ff-6efa-4d5b-9988-5afcbb61a9cd', 'age': 51, 'annual_income': 123864.0}>>
<Record c=<Node element_id='4:7ee77186-f382-4f2a-ad67-d6d490e9527b:3' labels=frozenset({'Client'}) properties={'loan_type': 'Home Loan', 'client_id': '1f0e4b4a-886c-4a30-9c26-ffa8ccce240c', 'age': 46, 'annual_income': 123296.0

In [ ]:
import pandas as pd
from neo4j import GraphDatabase
import uuid # For generating unique IDs

# --- Assume your DataFrames are already loaded ---
# Example data (replace with your actual DataFrame loading)
# clients_data = {
#     'client_id': ['f3a3c571-7476-4899-b5a3-adb3254a9493', '649dda6e-b49c-43dc-acbc-408cc5521660', '81c1e7ff-6efa-4d5b-9988-5afcbb61a9cd', '1f0e4b4a-886c-4a30-9c26-ffa8ccce240c', '436f40f2-74b8-4e87-bc99-1c531484f407'],
#     'age': [34, 45, 51, 46, 33],
#     'annual_income': [160462, 72662, 123864, 123296, 53462],
#     'loan_type': ['Auto Loan', 'Home Loan', 'Personal Loan', 'Home Loan', 'Home Loan'] # This can be a primary loan type
# }
# clients_df = pd.DataFrame(clients_data)

# transactions_data = {
#     'client_id': ['f3a3c571-7476-4899-b5a3-adb3254a9493', 'f3a3c571-7476-4899-b5a3-adb3254a9493', 'f3a3c571-7476-4899-b5a3-adb3254a9493', 'f3a3c571-7476-4899-b5a3-adb3254a9493', 'f3a3c571-7476-4899-b5a3-adb3254a9493'],
#     'date': ['2023-01-01', '2023-01-01', '2023-01-01', '2023-01-01', '2023-01-01'],
#     'amount': [13371.83, -222.27, -156.99, -164.54, -223.65],
#     'type': ['credit', 'debit', 'debit', 'debit', 'debit'],
#     'category': ['salary', 'shopping', 'groceries', 'groceries', 'entertainment']
# }
# transactions_df = pd.DataFrame(transactions_data)
# # Ensure date column is in datetime format if not already
# transactions_df['date'] = pd.to_datetime(transactions_df['date'])


# loans_data = {
#     'client_id': ['f3a3c571-7476-4899-b5a3-adb3254a9493', '649dda6e-b49c-43dc-acbc-408cc5521660', '81c1e7ff-6efa-4d5b-9988-5afcbb61a9cd', '1f0e4b4a-886c-4a30-9c26-ffa8ccce240c', '436f40f2-74b8-4e87-bc99-1c531484f407'],
#     'loan_type': ['Auto Loan', 'Home Loan', 'Personal Loan', 'Home Loan', 'Home Loan'],
#     'loan_amount': [414112, 400343, 261115, 476002, 324083],
#     'emi': [18004.87, 13804.93, 9325.54, 22666.76, 11574.39]
# }
# loans_df = pd.DataFrame(loans_data)

# --- Neo4j Connection Details ---
# Replace with your Neo4j AuraDB URI or local instance URI
uri = "neo4j+s://e14248e5.databases.neo4j.io"  # Or "neo4j://localhost:7687"
username = "neo4j"
password = "j_OfoqhHCpE1vq-qFEHfcyI0WkjIKcGktFmqGtnp0Oc"

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def execute_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return [record for record in result]

    def execute_write_query(self, query, parameters=None):
        with self.driver.session() as session:
            session.write_transaction(self._execute_transaction_fn, query, parameters)

    @staticmethod
    def _execute_transaction_fn(tx, query, parameters):
        tx.run(query, parameters)

def create_constraints(conn):
    """Creates unique constraints in Neo4j."""
    queries = [
        "CREATE CONSTRAINT client_id_unique IF NOT EXISTS FOR (c:Client) REQUIRE c.clientId IS UNIQUE;",
        "CREATE CONSTRAINT category_name_unique IF NOT EXISTS FOR (cat:Category) REQUIRE cat.name IS UNIQUE;",
        "CREATE CONSTRAINT loan_type_name_unique IF NOT EXISTS FOR (lt:LoanType) REQUIRE lt.name IS UNIQUE;",
        "CREATE CONSTRAINT transaction_id_unique IF NOT EXISTS FOR (t:Transaction) REQUIRE t.transactionId IS UNIQUE;",
        "CREATE CONSTRAINT loan_id_unique IF NOT EXISTS FOR (l:Loan) REQUIRE l.loanId IS UNIQUE;",
        "CREATE CONSTRAINT monthly_summary_id_unique IF NOT EXISTS FOR (ms:MonthlySummary) REQUIRE ms.summaryId IS UNIQUE;"
    ]
    for query in queries:
        try:
            conn.execute_write_query(query)
            print(f"Successfully executed: {query.split('FOR')[0]}...") # Shortened log
        except Exception as e:
            print(f"Error creating constraint with query '{query.split('FOR')[0]}...': {e}")
    print("Constraints check/creation process finished.")


def load_clients(conn, df):
    """Loads client data from DataFrame into Neo4j."""
    print("Loading clients...")
    for index, row in df.iterrows():
        query = """
        MERGE (c:Client {clientId: $client_id})
        ON CREATE SET
            c.age = $age,
            c.annualIncome = $annual_income
        """
        parameters = {
            "client_id": row["client_id"],
            "age": int(row["age"]),
            "annual_income": float(row["annual_income"])
        }
        conn.execute_write_query(query, parameters)
    print(f"{len(df)} clients processed.")

def load_loan_types(conn, df):
    """Loads unique loan types from loans_df into Neo4j."""
    print("Loading loan types...")
    unique_loan_types = df['loan_type'].unique()
    for loan_type_name in unique_loan_types:
        query = "MERGE (lt:LoanType {name: $name})"
        parameters = {"name": loan_type_name}
        conn.execute_write_query(query, parameters)
    print(f"{len(unique_loan_types)} unique loan types processed.")

def load_loans(conn, df):
    """Loads loan data and links them to clients and loan types."""
    print("Loading loans...")
    for index, row in df.iterrows():
        # Create a unique ID for the loan.
        # Assumes a client has only one loan of a specific type.
        # If a client can have multiple 'Home Loans', a more robust unique ID (e.g., using uuid) is needed.
        loan_id = f"{row['client_id']}_{row['loan_type']}"

        query = """
        MATCH (c:Client {clientId: $client_id})
        MATCH (lt:LoanType {name: $loan_type_name})
        MERGE (l:Loan {loanId: $loan_id})
        ON CREATE SET
            l.loanAmount = $loan_amount,
            l.emi = $emi
        MERGE (c)-[:HAS_LOAN]->(l)
        MERGE (l)-[:IS_OF_TYPE]->(lt)
        """
        parameters = {
            "client_id": row["client_id"],
            "loan_type_name": row["loan_type"],
            "loan_id": loan_id,
            "loan_amount": float(row["loan_amount"]),
            "emi": float(row["emi"])
        }
        conn.execute_write_query(query, parameters)
    print(f"{len(df)} loans processed.")

def load_transaction_categories(conn, df):
    """Loads unique transaction categories from transactions_df into Neo4j."""
    print("Loading transaction categories...")
    unique_categories = df['category'].unique()
    for category_name in unique_categories:
        query = "MERGE (cat:Category {name: $name})"
        parameters = {"name": category_name}
        conn.execute_write_query(query, parameters)
    print(f"{len(unique_categories)} unique transaction categories processed.")

def load_transactions(conn, df):
    """Loads transaction data and links them to clients and categories."""
    print("Loading transactions...")
    for index, row in df.iterrows():
        # Generate a unique ID for the transaction
        transaction_id = str(uuid.uuid4())
        
        # Ensure date is in 'YYYY-MM-DD' string format for Neo4j date() function
        transaction_date_str = row["date"].strftime('%Y-%m-%d')

        query = """
        MATCH (c:Client {clientId: $client_id})
        MATCH (cat:Category {name: $category_name})
        MERGE (t:Transaction {transactionId: $transaction_id})
        ON CREATE SET
            t.date = date($date_str),
            t.amount = $amount,
            t.type = $type
        MERGE (c)-[:MADE_TRANSACTION]->(t)
        MERGE (t)-[:BELONGS_TO_CATEGORY]->(cat)
        """
        parameters = {
            "client_id": row["client_id"],
            "category_name": row["category"],
            "transaction_id": transaction_id,
            "date_str": transaction_date_str,
            "amount": float(row["amount"]),
            "type": row["type"]
        }
        conn.execute_write_query(query, parameters)
    print(f"{len(df)} transactions processed.")

def aggregate_monthly_summaries(conn):
    """Aggregates monthly income, expenses, and savings for each client."""
    print("Aggregating monthly summaries...")
    query = """
    MATCH (c:Client)-[:MADE_TRANSACTION]->(t:Transaction)
    WITH c, t.date.year AS year, t.date.month AS month,
         SUM(CASE WHEN t.type = 'credit' THEN t.amount ELSE 0 END) AS monthlyIncome,
         SUM(CASE WHEN t.type = 'debit' THEN t.amount ELSE 0 END) AS monthlyRawExpense // Expenses are negative
    MERGE (ms:MonthlySummary {summaryId: c.clientId + '_' + year + '-' + month }) // Unique ID for the summary
    ON CREATE SET
        ms.year = year,
        ms.month = month,
        ms.totalIncome = monthlyIncome,
        ms.totalExpenses = monthlyRawExpense,
        ms.netSavings = monthlyIncome + monthlyRawExpense
    ON MATCH SET // If you re-run, it updates existing summaries
        ms.totalIncome = monthlyIncome,
        ms.totalExpenses = monthlyRawExpense,
        ms.netSavings = monthlyIncome + monthlyRawExpense
    MERGE (c)-[:HAS_MONTHLY_SUMMARY]->(ms)
    RETURN count(ms) as summaries_created_or_updated;
    """
    result = conn.execute_query(query) # Using execute_query as it returns a result
    if result:
        print(f"{result[0]['summaries_created_or_updated']} monthly summaries created or updated.")
    else:
        print("No monthly summaries to create or update.")


if __name__ == "__main__":
    # Establish connection
    NEO4J_URI = "neo4j+s://e14248e5.databases.neo4j.io"  # Or "neo4j://localhost:7687"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "j_OfoqhHCpE1vq-qFEHfcyI0WkjIKcGktFmqGtnp0Oc"
    db_conn = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    print("Successfully connected to Neo4j.")

    try:
        # Create constraints (idempotent)
        create_constraints(db_conn)

        # Load data from DataFrames
        # load_clients(db_conn, clients_df)
        # load_loan_types(db_conn, loans_df) # Needs to be run before load_loans
        # load_loans(db_conn, loans_df)
        # load_transaction_categories(db_conn, transactions_df) # Needs to be run before load_transactions
        load_transactions(db_conn, transactions_df)

        # Aggregate financial summaries
        aggregate_monthly_summaries(db_conn)

        print("\nData loading and aggregation complete.")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the connection
        db_conn.close()
        print("Neo4j connection closed.")


Successfully connected to Neo4j.


/var/folders/wl/8jx7ngdj365dkn33jz1rs3180000gn/T/ipykernel_22617/1261644342.py:55: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._execute_transaction_fn, query, parameters)


Successfully executed: CREATE CONSTRAINT client_id_unique IF NOT EXISTS ...
Successfully executed: CREATE CONSTRAINT category_name_unique IF NOT EXISTS ...
Successfully executed: CREATE CONSTRAINT loan_type_name_unique IF NOT EXISTS ...
Successfully executed: CREATE CONSTRAINT transaction_id_unique IF NOT EXISTS ...
Successfully executed: CREATE CONSTRAINT loan_id_unique IF NOT EXISTS ...
Successfully executed: CREATE CONSTRAINT monthly_summary_id_unique IF NOT EXISTS ...
Constraints check/creation process finished.
Loading transactions...


/var/folders/wl/8jx7ngdj365dkn33jz1rs3180000gn/T/ipykernel_22617/1261644342.py:55: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._execute_transaction_fn, query, parameters)
